# Projeto: Dashboard Olist

## Objetivo:

Este notebook visa realizar o tratamento e pré-processamento de dados, preparando-os para a subsequente construção de um dashboard no Power BI. Essa etapa é desenvolvida em Python, utilizando a biblioteca Pandas, devido ao seu vasto potencial de processamento de dados. A intenção é preparar os dados de forma a facilitar a exportação posterior em formato CSV. Dessa maneira, otimizamos o trabalho que teríamos no Power Query e garantimos que nosso arquivo .pbix permaneça mais leve.

## Dados:

Trata-se de uma base de dados pública do comércio eletrônico brasileiro, que registra pedidos efetuados na loja Olist. O conjunto de dados compreende informações de 100 mil pedidos realizados entre 2016 e 2018, abrangendo diversos marketplaces no Brasil. Suas características possibilitam a visualização de um pedido sob diversas perspectivas, desde o status do pedido, preço, desempenho de pagamento e frete, até a localização do cliente, atributos do produto e, por fim, avaliações redigidas pelos clientes. Adicionalmente, disponibilizamos um conjunto de dados de geolocalização que relaciona os CEPs brasileiros às coordenadas de latitude/longitude.

Os dados encontram-se disponíveis no [Kaggle.](https://www.kaggle.com/datasets/olistbr/brazilian-ecommerce)


## Importando Bibliotecas

In [1]:
import pandas as pd
import numpy as np
import datetime
import warnings
import os

# configurando pandas para mostrar todas as linhas e colunas
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None) 

# configurando pandas para não mostrar notação científica para números

warnings.filterwarnings("ignore")

## Importando Dados

In [2]:
# Criando Dataframes a partis dos aquivos CSV
customers_df = pd.read_csv("./data/olist_customers_dataset.csv")
geo_df = pd.read_csv("./data/olist_geolocation_dataset.csv")
orderitem_df = pd.read_csv("./data/olist_order_items_dataset.csv")
orderpay_df = pd.read_csv("./data/olist_order_payments_dataset.csv")
orderreviews_df = pd.read_csv("./data/olist_order_reviews_dataset.csv")
orders_df = pd.read_csv("./data/olist_orders_dataset.csv")
products_df = pd.read_csv("./data/olist_products_dataset.csv")
sellers_df = pd.read_csv("./data/olist_sellers_dataset.csv")

In [3]:
# Vendo as 10 linhas do arquivo de clientes
customers_df.head(10)

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP
5,879864dab9bc3047522c92c82e1212b8,4c93744516667ad3b8f1fb645a3116a4,89254,jaragua do sul,SC
6,fd826e7cf63160e536e0908c76c3f441,addec96d2e059c80c30fe6871d30d177,4534,sao paulo,SP
7,5e274e7a0c3809e14aba7ad5aae0d407,57b2a98a409812fe9618067b6b8ebe4f,35182,timoteo,MG
8,5adf08e34b2e993982a47070956c5c65,1175e95fb47ddff9de6b2b06188f7e0d,81560,curitiba,PR
9,4b7139f34592b3a31687243a302fa75b,9afe194fb833f79e300e37e580171f22,30575,belo horizonte,MG


In [4]:
# Quantas linhas e colunas tem o dataframe de clientes
customers_df.shape

(99441, 5)

## "Cheirando" os Dados

Além de conhecer os metadados, temos também que dar uma "cheiradinha" neles! Visualizar algumas linhas, avaliar os tipos de colunas e como as diferentes bases se relacionam.

In [5]:
# Olhando as 5 primeiras linhas de todos os dataframes

print("customers_df:")
display(customers_df.head())

print("\ngeo_df:")
display(geo_df.head())

print("\norderitem_df:")
display(orderitem_df.head())

print("\norderpay_df:")
display(orderpay_df.head())

print("\norderreviews_df:")
display(orderreviews_df.head())

print("\norders_df:")
display(orders_df.head())

print("\nproducts_df:")
display(products_df.head())

print("\nsellers_df:")
display(sellers_df.head())


customers_df:


,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP



geo_df:


,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state
0,1037,-23.545621,-46.639292,sao paulo,SP
1,1046,-23.546081,-46.644820,sao paulo,SP
2,1046,-23.546129,-46.642951,sao paulo,SP
3,1041,-23.544392,-46.639499,sao paulo,SP
4,1035,-23.541578,-46.641607,sao paulo,SP



orderitem_df:


...

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00



products_df:


,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumaria,40.0,287.0,1.0,225.0,16.0,10.0,14.0
1,3aa071139cb16b67ca9e5dea641aaa2f,artes,44.0,276.0,1.0,1000.0,30.0,18.0,20.0
2,96bd76ec8810374ed1b65e291975717f,esporte_lazer,46.0,250.0,1.0,154.0,18.0,9.0,15.0
3,cef67bcfe19066a932b7673e239eb23d,bebes,27.0,261.0,1.0,371.0,26.0,4.0,26.0
4,9dc1a7de274444849c219cff195d0b71,utilidades_domesticas,37.0,402.0,4.0,625.0,20.0,17.0,13.0



sellers_df:


,seller_id,seller_zip_code_prefix,seller_city,seller_state
0,3442f8959a84dea7ee197c632cb2df15,13023,campinas,SP
1,d1b65fc7debc3361ea86b5f14c68d2e2,13844,mogi guacu,SP
2,ce3ad9de960102d0677a81f5d0bb7b2d,20031,rio de janeiro,RJ
3,c0f3eea2e14555b6faeea3dd58c1b1c3,4195,sao paulo,SP
4,51a04a8a6bdcb23deccc82b0b80742cf,12914,braganca paulista,SP


In [6]:
# Olhando as informações de todos os dataframes

print("customers_df:\n")
display(customers_df.info())

print("\ngeo_df:\n")
display(geo_df.info())

print("\norderitem_df:\n")
display(orderitem_df.info())

print("\norderpay_df:\n")
display(orderpay_df.info())

print("\norderreviews_df:\n")
display(orderreviews_df.info())

print("\norders_df:\n")
display(orders_df.info())

print("\nproducts_df:\n")
display(products_df.info())

print("\nsellers_df:\n")
display(sellers_df.info())


customers_df:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 5 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   customer_id               99441 non-null  object
 1   customer_unique_id        99441 non-null  object
 2   customer_zip_code_prefix  99441 non-null  int64 
 3   customer_city             99441 non-null  object
 4   customer_state            99441 non-null  object
dtypes: int64(1), object(4)
memory usage: 3.8+ MB


None


geo_df:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000163 entries, 0 to 1000162
Data columns (total 5 columns):
 #   Column                       Non-Null Count    Dtype  
---  ------                       --------------    -----  
 0   geolocation_zip_code_prefix  1000163 non-null  int64  
 1   geolocation_lat              1000163 non-null  float64
 2   geolocation_lng              1000163 non-null  float64
 3   geolocation_city             1000163 non-null  object 
 4   geolocation_state            1000163 non-null  object 
dtypes: float64(2), int64(1), object(2)
memory usage: 38.2+ MB


None


orderitem_df:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112650 entries, 0 to 112649
Data columns (total 7 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   order_id             112650 non-null  object 
 1   order_item_id        112650 non-null  int64  
 2   product_id           112650 non-null  object 
 3   seller_id            112650 non-null  object 
 4   shipping_limit_date  112650 non-null  object 
 5   price                112650 non-null  float64
 6   freight_value        112650 non-null  float64
dtypes: float64(2), int64(1), object(4)
memory usage: 6.0+ MB


...

None


products_df:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32951 entries, 0 to 32950
Data columns (total 9 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   product_id                  32951 non-null  object 
 1   product_category_name       32341 non-null  object 
 2   product_name_lenght         32341 non-null  float64
 3   product_description_lenght  32341 non-null  float64
 4   product_photos_qty          32341 non-null  float64
 5   product_weight_g            32949 non-null  float64
 6   product_length_cm           32949 non-null  float64
 7   product_height_cm           32949 non-null  float64
 8   product_width_cm            32949 non-null  float64
dtypes: float64(7), object(2)
memory usage: 2.3+ MB


None


sellers_df:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3095 entries, 0 to 3094
Data columns (total 4 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   seller_id               3095 non-null   object
 1   seller_zip_code_prefix  3095 non-null   int64 
 2   seller_city             3095 non-null   object
 3   seller_state            3095 non-null   object
dtypes: int64(1), object(3)
memory usage: 96.8+ KB


None

# Tratamento/Pré-Processamento dos Dados

### Customers_df 

- Vamos Alterar as Siglas dos Estados dos Clientes para o nome completo. Assim fica melhor para o Power Bi identificar a geolocalização.

In [7]:
customers_df.head()

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP


In [8]:
# Dados nulos
porcentagem_nulos = (customers_df.isnull().mean() * 100).round(2)

print(porcentagem_nulos)

customer_id                 0.0
customer_unique_id          0.0
customer_zip_code_prefix    0.0
customer_city               0.0
customer_state              0.0
dtype: float64


In [9]:
# Vendo os valores únicos
valores_unicos = customers_df['customer_state'].unique()

# Exibindo os valores únicos
print(valores_unicos)

['SP' 'SC' 'MG' 'PR' 'RJ' 'RS' 'PA' 'GO' 'ES' 'BA' 'MA' 'MS' 'CE' 'DF'
 'RN' 'PE' 'MT' 'AM' 'AP' 'AL' 'RO' 'PB' 'TO' 'PI' 'AC' 'SE' 'RR']


In [10]:
customers_df['customer_state'] = customers_df['customer_state'].map({
    'AC': 'Acre',
    'AL': 'Alagoas',
    'AP': 'Amapá',
    'AM': 'Amazonas',
    'BA': 'Bahia',
    'CE': 'Ceará',
    'DF': 'Distrito Federal',
    'ES': 'Espírito Santo',
    'GO': 'Goiás',
    'MA': 'Maranhão',
    'MT': 'Mato Grosso',
    'MS': 'Mato Grosso do Sul',
    'MG': 'Minas Gerais',
    'PA': 'Pará',
    'PB': 'Paraíba',
    'PR': 'Paraná',
    'PE': 'Pernambuco',
    'PI': 'Piauí',
    'RJ': 'Rio de Janeiro',
    'RN': 'Rio Grande do Norte',
    'RS': 'Rio Grande do Sul',
    'RO': 'Rondônia',
    'RR': 'Roraima',
    'SC': 'Santa Catarina',
    'SP': 'São Paulo',
    'SE': 'Sergipe',
    'TO': 'Tocantins'
    })

customers_df.head()

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,São Paulo
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,São Paulo
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,São Paulo
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,São Paulo
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,São Paulo


### Geo_df

In [11]:
geo_df.head()

,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state
0,1037,-23.545621,-46.639292,sao paulo,SP
1,1046,-23.546081,-46.644820,sao paulo,SP
2,1046,-23.546129,-46.642951,sao paulo,SP
3,1041,-23.544392,-46.639499,sao paulo,SP
4,1035,-23.541578,-46.641607,sao paulo,SP


- Aqui não faremos alterações, pois uaremos a geolocalização dos estados do df customer_df.

### Orderitem_df

In [12]:
orderitem_df.head()

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14


In [13]:
# Dados nulos
porcentagem_nulos = (orderitem_df.isnull().mean() * 100).round(2)

print(porcentagem_nulos)

order_id               0.0
order_item_id          0.0
product_id             0.0
seller_id              0.0
shipping_limit_date    0.0
price                  0.0
freight_value          0.0
dtype: float64


- Aqui iremos formatar a data e hora da data limite de entrega

In [14]:
# Formatando para DD/MM/AAAA
orderitem_df['shipping_limit_date'] = pd.to_datetime(orderitem_df['shipping_limit_date']).dt.strftime('%d/%m/%Y')

In [15]:
orderitem_df.head()

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,19/09/2017,58.90,13.29
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,03/05/2017,239.90,19.93
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,18/01/2018,199.00,17.87
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,15/08/2018,12.99,12.79
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,13/02/2017,199.90,18.14


### Orderpay_df

In [16]:
orderpay_df.head()

,order_id,payment_sequential,payment_type,payment_installments,payment_value
0,b81ef226f3fe1789b1e8b2acac839d17,1,credit_card,8,99.33
1,a9810da82917af2d9aefd1278f1dcfa0,1,credit_card,1,24.39
2,25e8ea4e93396b6fa0d3dd708e76c1bd,1,credit_card,1,65.71
3,ba78997921bbcdc1373bb41e913ab953,1,credit_card,8,107.78
4,42fdf880ba16b47b59251dd489d4441a,1,credit_card,2,128.45


In [17]:
# Dados nulos
porcentagem_nulos = (orderpay_df.isnull().mean() * 100).round(2)

print(porcentagem_nulos)

order_id                0.0
payment_sequential      0.0
payment_type            0.0
payment_installments    0.0
payment_value           0.0
dtype: float64


- Aqui iremos formatar a coluna de tipos de pagamento para melhor compreensão

In [18]:
# Mapeando nomes
orderpay_df['payment_type'] = orderpay_df['payment_type'].map({
    'credit_card': 'Crédito',
    'voucher': 'Voucher',
    'boleto': 'Boleto',
    'debit_card' : 'Débito'
})
orderpay_df.head()

,order_id,payment_sequential,payment_type,payment_installments,payment_value
0,b81ef226f3fe1789b1e8b2acac839d17,1,Crédito,8,99.33
1,a9810da82917af2d9aefd1278f1dcfa0,1,Crédito,1,24.39
2,25e8ea4e93396b6fa0d3dd708e76c1bd,1,Crédito,1,65.71
3,ba78997921bbcdc1373bb41e913ab953,1,Crédito,8,107.78
4,42fdf880ba16b47b59251dd489d4441a,1,Crédito,2,128.45


### Orderreviews_df

In [19]:
orderreviews_df.head()

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
0,7bc2406110b926393aa56f80a40eba40,73fc7af87114b39712e6da79b0a377eb,4,NaN,NaN,2018-01-18 00:00:00,2018-01-18 21:46:59
1,80e641a11e56f04c1ad469d5645fdfde,a548910a1c6147796b98fdf73dbeba33,5,NaN,NaN,2018-03-10 00:00:00,2018-03-11 03:05:13
2,228ce5500dc1d8e020d8d1322874b6f0,f9e4b658b201a9f2ecdecbb34bed034b,5,NaN,NaN,2018-02-17 00:00:00,2018-02-18 14:36:24
3,e64fb393e7b32834bb789ff8bb30750e,658677c97b385a9be170737859d3511b,5,NaN,Recebi bem antes do prazo estipulado.,2017-04-21 00:00:00,2017-04-21 22:02:06
4,f7c4243c7fe1938f181bec41a392bdeb,8e6bfb81e283fa7e4f11123a3fb894f1,5,NaN,Parabéns lojas lannister adorei comprar pela I...,2018-03-01 00:00:00,2018-03-02 10:26:53


In [20]:
# Dados nulos
porcentagem_nulos = (orderreviews_df.isnull().mean() * 100).round(2)

print(porcentagem_nulos)

review_id                   0.00
order_id                    0.00
review_score                0.00
review_comment_title       88.34
review_comment_message     58.70
review_creation_date        0.00
review_answer_timestamp     0.00
dtype: float64


- Por conter muitos dados nulos, e tambpem porque não usaremos em nosso painel, iremos deletar a coluna 'eview_comment_title';

- A coluna review_comment_message pode ser util em nossa análise apser do número de dados nulos;


- Iremos deletar as colunas de data, por não serem necessárias em nossa análise


- Iremos renomear os valores nulos da coluna review_comment_message para 'nenhum comentário'

In [21]:
# Deletando colunas que não iremos usar
orderreviews_df.drop(['review_comment_title', 'review_creation_date', 'review_answer_timestamp'], axis=1, inplace=True)

In [22]:
# Substituindo valores nulos
orderreviews_df['review_comment_message'] = orderreviews_df['review_comment_message'].fillna('Nenhum Comentário')

In [23]:
orderreviews_df.head()

,review_id,order_id,review_score,review_comment_message
0,7bc2406110b926393aa56f80a40eba40,73fc7af87114b39712e6da79b0a377eb,4,Nenhum Comentário
1,80e641a11e56f04c1ad469d5645fdfde,a548910a1c6147796b98fdf73dbeba33,5,Nenhum Comentário
2,228ce5500dc1d8e020d8d1322874b6f0,f9e4b658b201a9f2ecdecbb34bed034b,5,Nenhum Comentário
3,e64fb393e7b32834bb789ff8bb30750e,658677c97b385a9be170737859d3511b,5,Recebi bem antes do prazo estipulado.
4,f7c4243c7fe1938f181bec41a392bdeb,8e6bfb81e283fa7e4f11123a3fb894f1,5,Parabéns lojas lannister adorei comprar pela I...


### Orders_df

In [24]:
orders_df.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00


In [25]:
# Dados nulos
porcentagem_nulos = (orders_df.isnull().mean() * 100).round(2)

print(porcentagem_nulos)

order_id                         0.00
customer_id                      0.00
order_status                     0.00
order_purchase_timestamp         0.00
order_approved_at                0.16
order_delivered_carrier_date     1.79
order_delivered_customer_date    2.98
order_estimated_delivery_date    0.00
dtype: float64


- Aqui formataremos as colunas com data;

- Nas colunas com data, manteremos apenas as referentes a Aprovação da Compra, Previsão de Entrega para o Cliente, e a de Entrega para o Cliente, pois assim poderemos saber o tempo de entrega, alem de saber a diferença entre variação do tempo de entrega com a previsão. Para isso, criaremos novas variaveis também (No power BI);

- Mapearemos os valores da coluna order_status para português

- Os valores nulos são referentes a compras não aprovadas, pedidos não entregues a transportadoras ou clientes. Por isso não há data

In [26]:
# Selecionando as colunas
date_columns = ['order_approved_at', 'order_delivered_customer_date', 'order_estimated_delivery_date', 'order_purchase_timestamp', 'order_delivered_carrier_date']
orders_df[date_columns] = orders_df[date_columns].apply(pd.to_datetime, errors='coerce')

# Formatando as datas para 'DD/MM/AAAA'
orders_df[date_columns] = orders_df[date_columns].apply(lambda x: x.dt.strftime('%d/%m/%Y'))

In [27]:
orders_df.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,02/10/2017,02/10/2017,04/10/2017,10/10/2017,18/10/2017
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,24/07/2018,26/07/2018,26/07/2018,07/08/2018,13/08/2018
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,08/08/2018,08/08/2018,08/08/2018,17/08/2018,04/09/2018
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,18/11/2017,18/11/2017,22/11/2017,02/12/2017,15/12/2017
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,13/02/2018,13/02/2018,14/02/2018,16/02/2018,26/02/2018


In [28]:
# Valores unicos do orders_df
valores_unicos = orders_df['order_status'].unique()

print(valores_unicos)

['delivered' 'invoiced' 'shipped' 'processing' 'unavailable' 'canceled'
 'created' 'approved']


In [29]:
# Traduzindo nomes
mapeamento = {
    'delivered': 'Entregue',
    'invoiced': 'Faturado',
    'shipped': 'Enviado',
    'processing': 'Processado',
    'unavailable': 'Indisponível',
    'canceled': 'Cancelado',
    'created': 'Criado',
    'approved': 'Aprovado'
}

orders_df['order_status'] = orders_df['order_status'].map(mapeamento)

In [30]:
orders_df.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,Entregue,02/10/2017,02/10/2017,04/10/2017,10/10/2017,18/10/2017
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,Entregue,24/07/2018,26/07/2018,26/07/2018,07/08/2018,13/08/2018
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,Entregue,08/08/2018,08/08/2018,08/08/2018,17/08/2018,04/09/2018
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,Entregue,18/11/2017,18/11/2017,22/11/2017,02/12/2017,15/12/2017
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,Entregue,13/02/2018,13/02/2018,14/02/2018,16/02/2018,26/02/2018


### Products_df

In [31]:
products_df.head()

,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumaria,40.0,287.0,1.0,225.0,16.0,10.0,14.0
1,3aa071139cb16b67ca9e5dea641aaa2f,artes,44.0,276.0,1.0,1000.0,30.0,18.0,20.0
2,96bd76ec8810374ed1b65e291975717f,esporte_lazer,46.0,250.0,1.0,154.0,18.0,9.0,15.0
3,cef67bcfe19066a932b7673e239eb23d,bebes,27.0,261.0,1.0,371.0,26.0,4.0,26.0
4,9dc1a7de274444849c219cff195d0b71,utilidades_domesticas,37.0,402.0,4.0,625.0,20.0,17.0,13.0


In [32]:
# Dados nulos
porcentagem_nulos = (products_df.isnull().mean() * 100).round(2)

print(porcentagem_nulos)

product_id                    0.00
product_category_name         1.85
product_name_lenght           1.85
product_description_lenght    1.85
product_photos_qty            1.85
product_weight_g              0.01
product_length_cm             0.01
product_height_cm             0.01
product_width_cm              0.01
dtype: float64


- Vamos alterar os nomes das categorias deprodutos para melhor compreensão

In [33]:
# Vendo quais são os nomes das categorias
valores_unicos = products_df['product_category_name'].unique()

# Exibindo os valores únicos
print(valores_unicos)

['perfumaria' 'artes' 'esporte_lazer' 'bebes' 'utilidades_domesticas'
 'instrumentos_musicais' 'cool_stuff' 'moveis_decoracao'
 'eletrodomesticos' 'brinquedos' 'cama_mesa_banho'
 'construcao_ferramentas_seguranca' 'informatica_acessorios'
 'beleza_saude' 'malas_acessorios' 'ferramentas_jardim'
 'moveis_escritorio' 'automotivo' 'eletronicos' 'fashion_calcados'
 'telefonia' 'papelaria' 'fashion_bolsas_e_acessorios' 'pcs'
 'casa_construcao' 'relogios_presentes'
 'construcao_ferramentas_construcao' 'pet_shop' 'eletroportateis'
 'agro_industria_e_comercio' nan 'moveis_sala' 'sinalizacao_e_seguranca'
 'climatizacao' 'consoles_games' 'livros_interesse_geral'
 'construcao_ferramentas_ferramentas' 'fashion_underwear_e_moda_praia'
 'fashion_roupa_masculina'
 'moveis_cozinha_area_de_servico_jantar_e_jardim'
 'industria_comercio_e_negocios' 'telefonia_fixa'
 'construcao_ferramentas_iluminacao' 'livros_tecnicos'
 'eletrodomesticos_2' 'artigos_de_festas' 'bebidas' 'market_place'
 'la_cuisine' 'const

In [34]:
products_df['product_category_name'] = products_df['product_category_name'].map({
'perfumaria' : 'Perfumaria',
'artes' : 'Artes',
'esporte_lazer' : 'Esporte e Lazer',
'bebes' : 'Bebes',
'utilidades_domesticas' : 'Utilidades Domesticas',
'instrumentos_musicais' : 'Instrumentos Musicais',
'cool_stuff' : 'Cool Stuff',
'moveis_decoracao' : 'Moveis e Decoracao',
'eletrodomesticos' : 'Eletrodomesticos',
'brinquedos' : 'Brinquedos',
'cama_mesa_banho' : 'Cama Mesa e Banho',
'construcao_ferramentas_seguranca' : 'Construcao, Ferramentas e Seguranca',
'informatica_acessorios' : 'Informatica e Acessorios',
'beleza_saude' : 'Beleza e Saude',
'malas_acessorios' : 'Malas e Acessorios',
'ferramentas_jardim' : 'Ferramentas de Jardim',
'moveis_escritorio' : 'Moveis de Escritorio',
'automotivo' : 'Automotivo',
'eletronicos' : 'Eletronicos',
'fashion_calcados' : 'Fashion e Calcados',
'telefonia' : 'Telefonia',
'papelaria' : 'Papelaria',
'fashion_bolsas_e_acessorios' : 'Fashion, Bolsas e Acessorios',
'pcs' : 'PCs',
'casa_construcao' : 'Casa e Construcao',
'relogios_presentes' : 'Relogios e Presentes',
'construcao_ferramentas_construcao' : 'Construção e Ferramentas',
'pet_shop' : 'Pet Shop',
'eletroportateis' : 'Eletroportateis',
'agro_industria_e_comercio' : 'Agro, Industria e Comercio',
'nan' : 'nan',
'moveis_sala' : 'Moveis para Sala',
'sinalizacao_e_seguranca' : 'Sinalizacao e Seguranca',
'climatizacao' : 'Climatizacao',
'consoles_games' : 'Consoles e Games',
'livros_interesse_geral' : 'Livros de Interesse Geral',
'construcao_ferramentas_ferramentas' : 'Construção e Ferramentas',
'fashion_underwear_e_moda_praia' : 'Fashion, Underwear e Moda Praia',
'fashion_roupa_masculina' : 'Fashion e Roupa Masculina',
'moveis_cozinha_area_de_servico_jantar_e_jardim' : 'Moveis de Cozinha, Area de Servico e Jantar e Jardim',
'industria_comercio_e_negocios' : 'Industria Comercio e Negocios',
'telefonia_fixa' : 'Telefonia Fixa',
'construcao_ferramentas_iluminacao' : 'Construcao, Ferramentas e Iluminacao',
'livros_tecnicos' : 'Livros Tecnicos',
'eletrodomesticos_2' : 'Eletrodomesticos 2',
'artigos_de_festas' : 'Artigos de Festas',
'bebidas' : 'Bebidas',
'market_place' : 'Market Place',
'la_cuisine' : 'La Cuisine',
'construcao_ferramentas_jardim' : 'Construcao, Ferramentas e Jardim',
'fashion_roupa_feminina' : 'Fashion e Roupa Feminina',
'casa_conforto' : 'Casa e Conforto',
'audio' : 'Audio',
'alimentos_bebidas' : 'Alimentos e Bebidas',
'musica' : 'Musica',
'alimentos' : 'Alimentos',
'tablets_impressao_imagem' : 'Tablets Impressao Imagem',
'livros_importados' : 'Livros Importados',
'portateis_casa_forno_e_cafe' : 'Portateis Casa Forno e Cafe',
'fashion_esporte' : 'Fashion Esporte',
'artigos_de_natal' : 'Artigos de Natal',
'fashion_roupa_infanto_juvenil' : 'Fashion Roupa Infanto Juvenil',
'dvds_blu_ray' : 'Dvds e Blu Ray',
'artes_e_artesanato' : 'Artes e Artesanato',
'pc_gamer' : 'PC Gamer',
'moveis_quarto' : 'Moveis para Quarto',
'cine_foto' : 'Cine e Foto',
'fraldas_higiene' : 'Fraldas e Higiene',
'flores' : 'Flores',
'casa_conforto_2' : 'Casa e Conforto 2',
'portateis_cozinha_e_preparadores_de_alimentos' : 'Portateis de Cozinha e Preparadores de Alimentos',
'seguros_e_servicos' : 'Seguros e Servicos',
'moveis_colchao_e_estofado' : 'Moveis, Colchao e Estofado',
'cds_dvds_musicais' : 'Cds, Dvds e Musicais'
})

products_df.head()

,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,1e9e8ef04dbcff4541ed26657ea517e5,Perfumaria,40.0,287.0,1.0,225.0,16.0,10.0,14.0
1,3aa071139cb16b67ca9e5dea641aaa2f,Artes,44.0,276.0,1.0,1000.0,30.0,18.0,20.0
2,96bd76ec8810374ed1b65e291975717f,Esporte e Lazer,46.0,250.0,1.0,154.0,18.0,9.0,15.0
3,cef67bcfe19066a932b7673e239eb23d,Bebes,27.0,261.0,1.0,371.0,26.0,4.0,26.0
4,9dc1a7de274444849c219cff195d0b71,Utilidades Domesticas,37.0,402.0,4.0,625.0,20.0,17.0,13.0


- Iremos criar uma nova variável com os cm3 dos produtos

In [35]:
# Criando
products_df['product_volume_cm3'] = (
    products_df['product_length_cm'] *
    products_df['product_height_cm'] *
    products_df['product_width_cm']
)

# Exibindo as primeiras linhas do DataFrame com a nova variável
print(products_df[['product_length_cm', 'product_height_cm', 'product_width_cm', 'product_volume_cm3']].head())

   product_length_cm  product_height_cm  product_width_cm  product_volume_cm3
0               16.0               10.0              14.0              2240.0
1               30.0               18.0              20.0             10800.0
2               18.0                9.0              15.0              2430.0
3               26.0                4.0              26.0              2704.0
4               20.0               17.0              13.0              4420.0


In [36]:
# Deletando colunas que não iremos usar
products_df.drop(['product_name_lenght', 'product_description_lenght', 'product_length_cm', 'product_height_cm', 'product_width_cm'], axis=1, inplace=True)

In [37]:
products_df.head()

,product_id,product_category_name,product_photos_qty,product_weight_g,product_volume_cm3
0,1e9e8ef04dbcff4541ed26657ea517e5,Perfumaria,1.0,225.0,2240.0
1,3aa071139cb16b67ca9e5dea641aaa2f,Artes,1.0,1000.0,10800.0
2,96bd76ec8810374ed1b65e291975717f,Esporte e Lazer,1.0,154.0,2430.0
3,cef67bcfe19066a932b7673e239eb23d,Bebes,1.0,371.0,2704.0
4,9dc1a7de274444849c219cff195d0b71,Utilidades Domesticas,4.0,625.0,4420.0


### Sellers_df


In [38]:
sellers_df.head()

,seller_id,seller_zip_code_prefix,seller_city,seller_state
0,3442f8959a84dea7ee197c632cb2df15,13023,campinas,SP
1,d1b65fc7debc3361ea86b5f14c68d2e2,13844,mogi guacu,SP
2,ce3ad9de960102d0677a81f5d0bb7b2d,20031,rio de janeiro,RJ
3,c0f3eea2e14555b6faeea3dd58c1b1c3,4195,sao paulo,SP
4,51a04a8a6bdcb23deccc82b0b80742cf,12914,braganca paulista,SP


Não faremos alterações

## Salvando

In [39]:
# Lista de DataFrames
dataframes = [customers_df, geo_df, orderitem_df, orderpay_df, orderreviews_df, orders_df, products_df, sellers_df]

# Salvando cada DataFrame em um arquivo CSV com o mesmo nome do DataFrame
for nome_variavel, df in zip(['customers_df', 'geo_df', 'orderitem_df', 'orderpay_df', 'orderreviews_df', 'orders_df', 'products_df', 'sellers_df'], dataframes):
    nome_arquivo = f'{nome_variavel}.csv'  # Nome do arquivo CSV usando o nome da variável que contém o DataFrame
    df.to_csv(nome_arquivo, index=False)

print("DataFrames salvos como arquivos CSV .")

DataFrames salvos como arquivos CSV .
